<a href="https://colab.research.google.com/github/Annemarie535257/Financial_ChatBot/blob/main/LLM_Financial_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate

In [ ]:
# Environment and configuration
import os
import time
import random
import math

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from transformers import (
    T5TokenizerFast,
    TFT5ForConditionalGeneration,
    create_optimizer,
)

import datasets as hf_datasets
import evaluate as hf_evaluate

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Paths
DATA_PATH = os.path.join(os.getcwd(), 'bitext-mortgage-loans-llm-chatbot-training-dataset.csv')
SAVE_ROOT = os.path.join(os.getcwd(), 'saved_models')
os.makedirs(SAVE_ROOT, exist_ok=True)

# Model and training hyperparameters
MODEL_NAME = 'google/flan-t5-small'
MAX_SOURCE_LENGTH = 256
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 3e-4
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01

RUN_ID = time.strftime('HUFI_V1_FLAN_T5_%Y%m%d_%H%M%S')
OUTPUT_DIR = os.path.join(SAVE_ROOT, RUN_ID)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'Run ID: {RUN_ID}\nSaving to: {OUTPUT_DIR}')


In [ ]:
# Load and inspect dataset
assert os.path.exists(DATA_PATH), f"Dataset not found at {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
print(df.head(2))
print('Columns:', df.columns.tolist())
print('Shape:', df.shape)

# Identify columns
QUESTION_COL = 'instruction' if 'instruction' in df.columns else df.columns[1]
ANSWER_COL = 'response' if 'response' in df.columns else df.columns[-1]

# Clean basic
for col in [QUESTION_COL, ANSWER_COL]:
    df[col] = df[col].astype(str).str.strip()

df = df.dropna(subset=[QUESTION_COL, ANSWER_COL])
df = df.drop_duplicates(subset=[QUESTION_COL, ANSWER_COL])
print('After cleaning:', df.shape)


In [ ]:
# Stratified train/val/test split by intent if available
from sklearn.model_selection import train_test_split

stratify_col = df['intent'] if 'intent' in df.columns else None
train_df, test_df = train_test_split(df, test_size=0.1, random_state=SEED, shuffle=True, stratify=stratify_col)
stratify_col_tv = train_df['intent'] if 'intent' in train_df.columns else None
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED, shuffle=True, stratify=stratify_col_tv)

print('Split sizes -> train:', len(train_df), 'val:', len(val_df), 'test:', len(test_df))


In [ ]:
# Tokenizer and formatting

tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
PREFIX = 'answer the question: '

def format_example(question: str, answer: str):
    return PREFIX + question, answer

for i in range(2):
    s, t = format_example(train_df.iloc[i][QUESTION_COL], train_df.iloc[i][ANSWER_COL])
    print('SRC:', s[:100])
    print('TGT:', t[:100])


In [ ]:
# Build Hugging Face datasets

def to_hf_dataset(frame: pd.DataFrame) -> hf_datasets.Dataset:
    sources, targets = [], []
    for _, row in frame.iterrows():
        s, t = format_example(row[QUESTION_COL], row[ANSWER_COL])
        sources.append(s)
        targets.append(t)
    return hf_datasets.Dataset.from_dict({'source': sources, 'target': targets})

raw_train = to_hf_dataset(train_df)
raw_val = to_hf_dataset(val_df)
raw_test = to_hf_dataset(test_df)

print(raw_train[0])


In [ ]:
# Tokenize datasets

def tokenize_function(batch):
    model_inputs = tokenizer(
        batch['source'],
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='np',
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['target'],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding='max_length',
            return_tensors='np',
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_tokenized = raw_train.map(tokenize_function, batched=True, remove_columns=['source','target'])
val_tokenized = raw_val.map(tokenize_function, batched=True, remove_columns=['source','target'])
test_tokenized = raw_test.map(tokenize_function, batched=True, remove_columns=['source','target'])

for ds in [train_tokenized, val_tokenized, test_tokenized]:
    ds.set_format(type='numpy')


In [ ]:
# tf.data input pipelines

def to_tf_dataset(tokenized: hf_datasets.Dataset, batch_size: int) -> tf.data.Dataset:
    feats = {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'labels': tokenized['labels'],
    }
    def gen():
        for i in range(len(tokenized)):
            yield {k: feats[k][i] for k in feats}
    sig = {
        'input_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32),
        'attention_mask': tf.TensorSpec(shape=(None,), dtype=tf.int32),
        'labels': tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
    return tf.data.Dataset.from_generator(gen, output_signature=sig).shuffle(1024, seed=SEED).batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = to_tf_dataset(train_tokenized, BATCH_SIZE)
val_ds = to_tf_dataset(val_tokenized, BATCH_SIZE)


In [ ]:
# Model and training loop

from google.colab import drive
drive.mount('/content/drive')

tf.keras.backend.clear_session()
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_NAME, from_pt=True)

num_train_steps = math.ceil(len(train_tokenized) / BATCH_SIZE) * EPOCHS
num_warmup_steps = int(num_train_steps * WARMUP_RATIO)

optimizer, lr_schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_warmup_steps=num_warmup_steps,
    num_train_steps=num_train_steps,
    weight_decay_rate=WEIGHT_DECAY,
)

model.compile(optimizer=optimizer)

# Change OUTPUT_DIR to a path in Google Drive
OUTPUT_DIR = os.path.join('/content/drive/MyDrive', 'saved_models', RUN_ID)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'Run ID: {RUN_ID}\nSaving to: {OUTPUT_DIR}')

ckpt_cb = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(OUTPUT_DIR, 'ckpt'),
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)

es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[ckpt_cb, es_cb],
)

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print('Saved to', OUTPUT_DIR)

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install rouge_score

In [ ]:
# Evaluation: BLEU, ROUGE-L, perplexity

bleu = hf_evaluate.load('sacrebleu')
rouge = hf_evaluate.load('rouge')

def generate_answers(questions, max_new_tokens=64):
    inputs = tokenizer(['answer the question: ' + q for q in questions], return_tensors='tf', padding=True, truncation=True, max_length=MAX_SOURCE_LENGTH)
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

sample_size = min(256, len(val_df))
sample_questions = val_df[QUESTION_COL].tolist()[:sample_size]
sample_refs = [[a] for a in val_df[ANSWER_COL].tolist()[:sample_size]]

preds = generate_answers(sample_questions)

bleu_res = bleu.compute(predictions=preds, references=sample_refs)
rouge_res = rouge.compute(predictions=preds, references=[r[0] for r in sample_refs])

val_loss = model.evaluate(val_ds, return_dict=True)['loss']
perplexity = math.exp(val_loss) if val_loss < 20 else float('inf')

print('BLEU:', bleu_res)
print('ROUGE-L:', rouge_res.get('rougeL'))
print('Val loss:', val_loss, 'Perplexity:', perplexity)


In [ ]:
# Qualitative test: predictions vs references on a small sample

NUM_EXAMPLES = 10
MAX_NEW_TOKENS = 64

sample_questions = val_df[QUESTION_COL].tolist()[:NUM_EXAMPLES]
sample_refs = val_df[ANSWER_COL].tolist()[:NUM_EXAMPLES]

def generate_answers_list(questions, max_new_tokens=64):
    inputs = tokenizer(
        ['answer the question: ' + q for q in questions],
        return_tensors='tf',
        padding=True,
        truncation=True,
        max_length=MAX_SOURCE_LENGTH,
    )
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

preds = generate_answers_list(sample_questions, max_new_tokens=MAX_NEW_TOKENS)

for i, (q, ref, pred) in enumerate(zip(sample_questions, sample_refs, preds), 1):
    print(f'--- Example {i} ---')
    print('Q:', q)
    print('REF:', ref[:400])
    print('PRED:', pred[:400])
    print()

print(f'Shown {NUM_EXAMPLES} qualitative examples.')
